## Comparez les résultats obtenus avec une approche basée sur la similarité des articles dans un espace vectoriel, à l'instar du calcul de similarité de l'approche item-item. La mesure de la similarité et la façon de l'utiliser pour estimer la pertinence d'articles similaires est laissé à votre discrétion.

Dans un premier temps on extrait la matrice

In [8]:
m = as.matrix(read.table("./citeseer.rtable"))

On va reutiliser les fonctions de cosinus de la correction du TP1

In [9]:
## Cosinus entre un vecteur v et chaque colonne dela matrice m
cosinus.vm <- function(v,m) { n <- sqrt(colSums(m^2)); (v %*% m)/(n * sqrt(sum(v^2))) }
## Cosinus des colonnes d'une matrice
cosinus.mm <- function(m) { n <- sqrt(colSums(m^2)); crossprod(m)/(n %o% n) }
# Trouve les indexes des premières 'n' valeurs maximales d'une matrice
max.nindex <- function(m, n=5) {
  i <- order(m, decreasing=TRUE)
  return(i[1:n])
}
min.nindex <- function(m, n=5) {
  i <- order(m)
  return(i[1:n])
}

On selectionne l'index de l'article 422908
On calcul la distance consinus du vecteur de l'article et de tout les autres articles
Les articles les plus proches de l'article 422908 seront considéré comme les articles à recommander

In [10]:
i <- grep('422908', as.character(rownames(m)), ignore.case=T)

sim.cos <- as.vector(cosinus.vm(m[,i], m))
i.sim.cos <- max.nindex(sim.cos)
sim.cos.sorted <- data.frame(Cos=sim.cos[i.sim.cos], Article=as.character(rownames(m)[i.sim.cos]), Index=i.sim.cos)
head(sim.cos.sorted)

Cos,Article,Index
<dbl>,<fct>,<int>
1.0000000,422908,747
0.5212860,96767,1081
0.4445542,496938,858
0.3680374,425638,755
0.3626593,522428,889


## Utilisez une validation croisée pour évaluer la performance de l'approche item-item.

On va utiliser une validation croisée de type leave-one-out. Il s'agit de la version extrème de la K-fold cross validation, où chaque vecteur sera considérer comme un ensemble de test comportant un unique élement. Nous allons donc essayer de prédire chaque vecteur de la matrice.

La première chose à faire est de récupérer chaque distance entre les articles. On se base sur la distance cosinus pour cela.

In [11]:
m.sim.cos = cosinus.mm(m)
head(m.sim.cos)

,X100299,X100967,X10151,X101705,X101863,X102458,X102886,X102966,X10302,X103700,⋯,X96767,X97060,X97150,X9721,X97410,X97863,X98185,X99113,X9947,X9993
X100299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X100967,NaN,1,0,NaN,0,NaN,0,NaN,0,0,⋯,0.0000000,0,NaN,0,0,NaN,0,NaN,0,0
X10151,NaN,0,1,NaN,0,NaN,0,NaN,0,0,⋯,0.0000000,0,NaN,0,0,NaN,0,NaN,0,0
X101705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X101863,NaN,0,0,NaN,1,NaN,0,NaN,0,0,⋯,0.2357023,0,NaN,0,0,NaN,0,NaN,0,0
X102458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On doit maintenant nettoyer un peu nos données. Par exemple la première colone ne comporte que des NaN, car elle ne comporte aucune référence vers d'autres articles.

Nous allons donc faire la somme de chaque colone et regarder si la somme des distances est suppérieur à 1. Cela permettra de selectionner seulement les articles qui possède au moins une ressemblance avec un autre article (la distance cosinus à lui même est 1. Si une autre distance à un article est définie alors la somme des distances sera suppérieur à 1)

In [12]:
index.m.sim.cos.noColNa <- which(colSums(m.sim.cos, na.rm = TRUE) > 1)

Une fois les index selectionnés, nous allons créer les matrices de base de notre prédiction:
- une matrice m "propre", ne comportant que les articles intérésant à analyser
- une matrice de prédiction que nous allons remplir avec chaque vecteur prédit
- une matrice comportant les distances cosinus des articles sélectionnés

In [13]:
m.clean <- m[index.m.sim.cos.noColNa,index.m.sim.cos.noColNa]

m.predicted <- matrix(, nrow=nrow(m.clean), ncol=ncol(m.clean))
rownames(m.predicted) <- rownames(m.clean)
colnames(m.predicted) <- colnames(m.clean)

m.clean.sim.cos <- m.sim.cos[index.m.sim.cos.noColNa,index.m.sim.cos.noColNa]

Un petit test de concepte:
- on sélectionne l'index du plus proche article d'un article en particulier en omettant lui-même

In [14]:
index = 1
index.closest <- max.nindex(m.clean.sim.cos[-index,index], 1)

-  on remplace ce vecteur dans la matrice de prédiction, là où était l'ancien vecteur réel.

In [17]:
m.predicted[,index] <- m.clean[,index.closest]
test.i2 <- data.frame(m.clean[,index], m.predicted[,index])
head(test.i2, 20)

,m.clean...index.,m.predicted...index.
,<int>,<int>
100967,0,0
10151,0,0
101863,0,0
102886,0,0
10302,0,0
103700,0,0
103914,0,0
10394,0,0
104047,0,0


On fait maintenant cela pour tout les articles par index de colone de la matrice

In [16]:
replace_by_closest <- function(index, m.predicted) {
  # on selectionne l'index du plus proche vecteur sans prendre en compte lui même
  index.closest <- max.nindex(m.clean.sim.cos[-index,index], 1)
  m.predicted[,index] <<- m.clean[,index.closest]
}

buffer <-lapply(1:ncol(m.clean), replace_by_closest, m.predicted)
RMSE <- sqrt(mean((m.clean - m.predicted)^2, na.rm=T))
print(RMSE)

[1] 0.07835628
